In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.cluster import KMeans

In [3]:
TEXT_COLOR = '#313131'
LINE_COLORS = ['#00A082', '#F2CC38', '#9B59B6', '#3498DB', '#F39C12']

sns.set(
    style='darkgrid', 
    rc={'figure.figsize':(6,4),
        'figure.dpi': 150,
        'figure.facecolor': 'w', 
        'legend.facecolor': 'w',
        'text.color': TEXT_COLOR,
        'font.family': 'Microsoft Sans Serif', # 'Open Sans',
        'axes.labelcolor': TEXT_COLOR,
        'xtick.color': TEXT_COLOR,
        'ytick.color': TEXT_COLOR}
)

sns.set_palette(sns.color_palette(LINE_COLORS))

# 1. Load the data

In [4]:
data = pd.read_csv('../data/house-prices-dataset/train.csv')

In [5]:
# Remove the outlier
data = data.drop(index=1298, axis=0)

In [6]:
# Selecting top-predictor columns IMO
cols = [
    'OverallQual', 
    'GrLivArea',
    'ExterQual',
    'GarageCars',
    'YearBuilt',
    'YearRemodAdd',
    'TotRmsAbvGrd',
    'Foundation',
    'Fireplaces',
    'FireplaceQu',
    'HeatingQC',
    'SalePrice'
]

In [7]:
data = data[cols]

In [8]:
def col_to_dummies(df, col):
    return pd.concat(
        [data, pd.get_dummies(data[col], prefix=col, drop_first=True)], 
        axis=1
    ).drop(col, axis=1)

In [9]:
data['FireplaceQu'] = data['FireplaceQu'].map({
    np.nan: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5
})

In [10]:
data['HeatingQC'] = data['HeatingQC'].map({
    'Po':0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4
})

In [11]:
data['ExterQual'] = data['ExterQual'].map({
    'Po':0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4
})

In [12]:
data = col_to_dummies(data, 'Foundation')

# 2. Explore how k-means works

### 2.1. Apply to raw data

In [ ]:
x_cols = data.columns.drop("SalePrice").tolist()

In [ ]:
km = KMeans(n_clusters=3, random_state=42)

In [ ]:
km.fit(data[x_cols])

In [ ]:
data['cluster'] = km.labels_

In [ ]:
sns.scatterplot(data=data, x='GrLivArea', y='SalePrice', hue='cluster', alpha=0.5)

Seems like GrLivArea was the most decisive factor to assign clusters because it is of large numbers.

### 2.2. Apply to scaled data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_data = data.drop("cluster", axis=1).copy()

In [ ]:
scaled_data[x_cols] = scaler.fit_transform(scaled_data[x_cols])

In [ ]:
km = KMeans(n_clusters=3, random_state=42)

In [ ]:
km.fit(scaled_data[x_cols])

In [ ]:
scaled_data['cluster'] = km.labels_

In [ ]:
sns.scatterplot(data=scaled_data, x='GrLivArea', y='SalePrice', hue='cluster', alpha=0.5)

### Plot principal components of PCA vs clusters

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

In [ ]:
pcomps = pca.fit_transform(scaled_data[x_cols])

In [ ]:
pc_cols = [f"PC{n+1}" for n in range(pcomps.shape[1])]

In [ ]:
scaled_data[pc_cols] = pcomps

In [ ]:
sns.scatterplot(data=scaled_data, x='PC1', y='PC2', hue='cluster', alpha=0.8)

In [ ]:
sns.scatterplot(data=scaled_data, x='PC1', y='SalePrice', hue='cluster', alpha=0.8)

### Apply k-means on principal components

In [ ]:
km = KMeans(n_clusters=3, random_state=42)

In [ ]:
km.fit(scaled_data[pc_cols])

In [ ]:
scaled_data['pca_cluster'] = km.labels_

In [ ]:
sns.scatterplot(data=scaled_data, x='PC1', y='PC2', hue='pca_cluster', alpha=0.8)

If we apply clustering with all principal compotents, we get the same resulting clustering labels.

# 3. Use k-means clusters and PCA components as features in cross-validation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.base import clone

In [ ]:
def perform_cross_validation(X, y, model, cv_folds=10, **kwargs):
    """Run cross-validation with preprocessing on a specified model."""
    original_model = clone(model)
    total_rows = X.shape[0]
    metric_track = None
    splits = generate_cv_splits(rows=total_rows, cv=cv_folds)

    for valid_start, valid_end in splits:
        train_idx, valid_idx = get_train_valid_idx(valid_start, valid_end, total_rows=total_rows)
        X_train, y_train, X_valid, y_valid = train_valid_split(X, y, train_idx, valid_idx)
        X_train, y_train, X_valid, y_valid = preprocess(X_train, X_valid, y_train, y_valid, **kwargs)

        # Re-instantiate the provided model
        model_ = clone(original_model)
        model_ = model_.fit(X_train, y_train)
        y_pred_log = model_.predict(X_valid)
        y_pred = np.exp(y_pred_log)

        # Evaluate and track performance
        current_metrics = pd.DataFrame(data=[evaluate_regression(y_valid, y_pred)])
        if metric_track is None:
            metric_track = current_metrics.copy()
        else:
            metric_track = pd.concat((metric_track, current_metrics), axis=0) # append the new result

    return dict(metric_track.mean())

In [ ]:
def generate_cv_splits(rows, cv=10):
    """Generate a list of start & end idx for cross validation."""
    step = rows // cv
    splits = []
    for split in range(0, cv):
        start = step*split
        end = start+step
        splits.append((start,  end))
    return splits

In [ ]:
def get_train_valid_idx(valid_start_idx, valid_end_idx, total_rows):
    """Transform validation start and end indexes into a list of training and validation indexes."""
    valid_idx = np.arange(valid_start_idx, valid_end_idx)
    all_idx = np.arange(total_rows)
    train_mask = np.isin(all_idx, valid_idx, invert=True)
    train_idx = all_idx[train_mask]
    return train_idx, valid_idx

In [ ]:
def train_valid_split(X, y, train_idx, valid_idx):
    """Split into train/test sets, based on index location."""
    X_train = X.copy().iloc[train_idx]
    y_train = y.copy().iloc[train_idx]
    X_valid = X.copy().iloc[valid_idx]
    y_valid = y.copy().iloc[valid_idx]
    return X_train, y_train, X_valid, y_valid

In [ ]:
def preprocess(X_train, X_valid, y_train, y_valid, kmeans=True, pca=True, target_log=True):
    """Ensures preprocessing without data leakage."""
    ncols = X_train.shape[1]
    
    X_train, X_valid, y_train, y_valid = X_train.values, X_valid.values, y_train.values, y_valid.values
    
    X_train, X_valid = scale(X_train, X_valid)
    
    if pca is True:
        X_train, X_valid = apply_pca(X_train, X_valid, ncols=ncols)
        
    if kmeans is True: 
        X_train, X_valid = apply_k_means(X_train, X_valid, ncols=ncols)
    
    if target_log:
        y_train = np.log(y_train)
    
    return X_train, y_train, X_valid, y_valid

In [ ]:
def scale(X_train, X_valid):
    """Fits X_train and transforms X_valid."""
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)
    return X_train, X_valid

In [ ]:
def apply_k_means(X_train, X_valid, ncols, n_clusters=3):
    """Fits X_train and predicts on X_valid."""
    km = KMeans(n_clusters=n_clusters, random_state=42)
    X_train_clusters = km.fit(X_train[:, :ncols]).labels_.reshape(-1, 1)
    X_valid_clusters = km.predict(X_valid[:, :ncols]).reshape(-1, 1)
    
    X_train = np.concatenate((X_train, X_train_clusters), axis=1)
    X_valid = np.concatenate((X_valid, X_valid_clusters), axis=1)

    return X_train, X_valid

In [ ]:
def apply_pca(X_train, X_valid, ncols, n_components=3):
    """Fits X_train and transforms X_valid."""
    pca = PCA(n_components=n_components)    
    pc_X_train = pca.fit_transform(X_train[:, :ncols])
    pc_X_valid = pca.transform(X_valid[:, :ncols])
    
    X_train = np.concatenate((X_train, pc_X_train), axis=1)
    X_valid = np.concatenate((X_valid, pc_X_valid), axis=1)
    
    return X_train, X_valid

In [ ]:
def evaluate_regression(y_test, y_pred, plot=False):
    """Evaluate performance based on MAE and RMSE."""
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    if plot is True:
        print(f'''MAE:\t{mae}\nRMSE:\t{rmse}''')
        plt.figure(figsize=(5,5))
        plt.scatter(y_test, y_pred, alpha=0.4)
        plt.plot(*2*[np.arange(0,500000)], color='r', ls='--')
        plt.xlabel('y true')
        plt.ylabel('y pred')
        plt.ylim(0, 500000)
        plt.xlim(0, 500000)
        plt.show()

    return {
        "rmse": rmse,
        "mae": mae,
    }

---

In [ ]:
X = data[x_cols].copy()

In [ ]:
y = data['SalePrice'].copy()

In [ ]:
cv_options = [
    {
        "kmeans": False, "pca": False,
    },
    {
        "kmeans": True, "pca": False,
    },
    {
        "kmeans": False, "pca": True,
    },
    {
        "kmeans": True, "pca": True,
    },
]

### 3.1 Use clusters as feature in a linear model

In [ ]:
model = LinearRegression()

In [ ]:
results = pd.DataFrame()

for kwargs in cv_options:
    print(kwargs)
    metrics = perform_cross_validation(X, y, model, cv_folds=10, **kwargs)
    result = pd.DataFrame(data=metrics, index=[str(kwargs)])
    results = pd.concat((results, result), axis=0)

In [ ]:
sns.heatmap(data=results, vmin=0, vmax=40000, annot=True, fmt='g', cmap='RdYlGn_r')
plt.title("Metric performance")

---

### 3.2 Use clusters as feature in a NON-linear model

In [ ]:
model = RandomForestRegressor(n_estimators=500, max_features=0.5, random_state=42)

In [ ]:
results = pd.DataFrame()

for kwargs in cv_options:
    metrics = perform_cross_validation(X, y, model, cv_folds=10, **kwargs)
    result = pd.DataFrame(data=metrics, index=[str(kwargs)])
    results = pd.concat((results, result), axis=0)

In [ ]:
sns.heatmap(data=results, vmin=0, vmax=40000, annot=True, fmt='g', cmap='RdYlGn_r')
plt.title("Metric performance")

---

# Conclusion
Linear regression explodes when using PCA as features!

Using k-means as feature improves performance minimally, at least for this specific data set.

---